In [1]:
%cd /ibex/user/slimhy/Shape2VecSet/code

/ibex/user/slimhy/Shape2VecSet/code


In [2]:
"""
Chained evaluation of the models.
"""
import argparse
import os.path as osp
import pprint
import json
import warnings

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import Sampler

import util.misc as misc
from util.misc import MetricLogger
from util.datasets import build_shape_surface_occupancy_dataset

import models.mlp_mapper as mlp_mapper
from models.mlp import MLP
from models.point_net import PointNet
from models.pointcloud_autoencoder import PointcloudAutoencoder

from losses.chamfer import chamfer_loss


def get_args_parser():
    parser = argparse.ArgumentParser("Performing Chained Eval", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_clip",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits", "graphedits_chained"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--chain_length",
        default=None,
        type=int,
        help="length of chains to load",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.",
    )
    parser.add_argument(
        "--alt_ae_embeds",
        type=str,
        default=None,
        help="Alternative autoencoder embeddings to use",
    )
    parser.add_argument(
        "--ft_bert",
        action="store_true",
        default=False,
        help="Also fine-tune the BERT model",
    )
    parser.add_argument(
        "--model",
        type=str,
        metavar="MODEL",
    )
    parser.add_argument(
        "--resume",
        default="",
        help="Resume from checkpoint"
    )
    parser.add_argument(
        "--resume_full_weights",
        action="store_true",
        default=False,
        help="Resume the full model weights with the EDM wrapper",
    )

    return parser

Jitting Chamfer 3D
Loaded JIT 3D CUDA chamfer distance


In [3]:
"""
xx mlp_mapper_bert_l1__256
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l1__256/checkpoint-50.pth

xx mlp_mapper_bert_direct_latent_256
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_direct_latent_256/checkpoint-50.pth

====================================

xx mlp_mapper_bert_bneck_1024_pcae_cpl
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_1024_pcae__fine_chained_cpl/checkpoint-59.pth

xx mlp_mapper_bert_bneck_512_pcae_cpl
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_512_pcae__fine_cpl__chained/checkpoint-59.pth

xx mlp_mapper_bert_bneck_256_pcae_cpl
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_256_pcae__fine_cpl__chained/checkpoint-59.pth

xx mlp_mapper_bert_l8_pcae_cpl
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l8_pcae__fine_cpl__chained/checkpoint-59.pth

xx mlp_mapper_bert_l4_pcae_cpl
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l4_pcae__fine_cpl__chained/checkpoint-59.pth

====================================
o mlp_mapper_bert_bneck_1024_pcae
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_1024_pcae__fine_chained/checkpoint-59.pth

o mlp_mapper_bert_bneck_512_pcae
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_512_pcae__fine_chained/checkpoint-59.pth

o mlp_mapper_bert_bneck_256_pcae
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_256_pcae__fine_chained/checkpoint-59.pth

o mlp_mapper_bert_l8_pcae
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l8_pcae__fine_chained/checkpoint-59.pth

xx mlp_mapper_bert_l4_pcae
/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l4_pcae__fine_chained/checkpoint-59.pth
"""

# Set dummy arg string to debug the parser
call_string = """--ae_pth /ibex/user/slimhy/Shape2VecSet/output/pc_ae/best_model.pt \
    --ae-latent-dim 256 \
    --text_model_name bert-base-uncased \
    --dataset graphedits_chained \
    --data_path /ibex/user/slimhy/ShapeWalk/ \
    --data_type release_chained \
    --num_workers 8 \
    --model mlp_mapper_bert_l8_pcae \
    --resume /ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l8_pcae__fine_chained/checkpoint-59.pth \
    --resume_full_weights \
    --device cuda \
    --fetch_keys \
    --use_embeds \
    --alt_ae_embeds pc_ae \
    --seed 0"""
# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())
args.use_clip = "clip" in args.text_model_name
device = torch.device(args.device)

model = mlp_mapper.__dict__[args.model](use_linear_proj=not args.use_clip)
model.to(device)

# Load the checkpoint
if args.resume:
    print("Loading checkpoint [%s]..." % args.resume)
    checkpoint = torch.load(args.resume, map_location="cpu")
    model.load_state_dict(checkpoint["model"])
    print("Done.")

Loading checkpoint [/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_l8_pcae__fine_chained/checkpoint-59.pth]...
Done.


In [4]:

class ChainSampler(Sampler):
    def __init__(self, dataset, batch_size, chain_length):
        self.dataset = dataset
        self.batch_size = batch_size
        self.chain_length = chain_length
        self.num_samples = len(dataset)
        self.n_batches = self.num_samples // (self.batch_size)
        self.n_chains = self.num_samples // self.chain_length

    def __iter__(self):
        t = 0
        for m in range(self.n_chains):
            for k in range(self.chain_length):
                for j in range(self.batch_size):
                    yield m*(self.chain_length*self.batch_size)+k+j*self.chain_length
                    t += 1
                    if t == self.num_samples:
                        return

    def __len__(self):
        return self.num_samples

def get_loader(args, batch_size, chain_length):
    args.batch_size = batch_size
    args.chain_length = chain_length

    dataset_val = build_shape_surface_occupancy_dataset("val", args=args)
    chain_sampler = ChainSampler(dataset_val, batch_size=args.batch_size, chain_length=args.chain_length)
    data_loader_val = torch.utils.data.DataLoader(
        dataset_val,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
        sampler=chain_sampler
    )

    return data_loader_val

In [5]:
# --------------------
# Fix the seed for reproducibility
seed = args.seed + misc.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)

cudnn.benchmark = True

args.fetch_keys = True
# --------------------

In [7]:
def describe_pc_ae(args):
    # Make an AE.
    if args.encoder_net == "pointnet":
        ae_encoder = PointNet(init_feat_dim=3, conv_dims=args.encoder_conv_layers)
        encoder_latent_dim = args.encoder_conv_layers[-1]
    else:
        raise NotImplementedError()

    if args.decoder_net == "mlp":
        ae_decoder = MLP(
            in_feat_dims=encoder_latent_dim,
            out_channels=args.decoder_fc_neurons + [args.n_pc_points * 3],
            b_norm=False,
        )

    model = PointcloudAutoencoder(ae_encoder, ae_decoder)
    return model


def load_state_dicts(checkpoint_file, map_location=None, **kwargs):
    """ Load torch items from saved state_dictionaries"""
    if map_location is None:
        checkpoint = torch.load(checkpoint_file)
    else:
        checkpoint = torch.load(checkpoint_file, map_location=map_location)

    for key, value in kwargs.items():
        value.load_state_dict(checkpoint[key])

    epoch = checkpoint.get('epoch')
    if epoch:
        return epoch


def read_saved_args(config_file, override_or_add_args=None, verbose=False):
    """
    :param config_file: json file containing arguments
    :param override_args: dict e.g., {'gpu': '0'} will set the resulting arg.gpu to be 0
    :param verbose:
    :return:
    """
    parser = argparse.ArgumentParser()
    args = parser.parse_args([])
    with open(config_file, "r") as f_in:
        args.__dict__ = json.load(f_in)

    if override_or_add_args is not None:
        for key, val in override_or_add_args.items():
            args.__setattr__(key, val)

    if verbose:
        args_string = pprint.pformat(vars(args))
        print(args_string)

    return args


def load_pretrained_pc_ae(model_file):
    config_file = osp.join(osp.dirname(model_file), "config.json.txt")
    pc_ae_args = read_saved_args(config_file)
    pc_ae = describe_pc_ae(pc_ae_args)

    if osp.join(pc_ae_args.log_dir, "best_model.pt") != osp.abspath(model_file):
        warnings.warn(
            "The saved best_model.pt in the corresponding log_dir is not equal to the one requested."
        )

    best_epoch = load_state_dicts(model_file, model=pc_ae)
    print(f"Pretrained PC-AE is loaded at epoch {best_epoch}.")
    return pc_ae, pc_ae_args

AE_MODEl_NAME = "pc_ae"
AE_MODEL_PATH = "/ibex/user/slimhy/Shape2VecSet/ckpt/pc_ae/best_model.pt"

# Instantiate autoencoder
print("Loading autoencoder [%s]..." % AE_MODEL_PATH)
pc_ae, pc_ae_args = load_pretrained_pc_ae(AE_MODEL_PATH)
pc_ae = pc_ae.to(device)
pc_ae = pc_ae.eval()
print("Done.")

Loading autoencoder [/ibex/user/slimhy/Shape2VecSet/ckpt/pc_ae/best_model.pt]...
Pretrained PC-AE is loaded at epoch 186.
Done.


/tmp/ipykernel_1132892/2777894551.py:64: UserWarning: The saved best_model.pt in the corresponding log_dir is not equal to the one requested.
  warnings.warn(


In [8]:
import torch.nn.functional as F

def apply_edit(net, x_a, embed_ab):
    # Reshape from (B, D, K) to (B, M)
    x_a = x_a.flatten(1)
    embed_ab = embed_ab.flatten(1)

    # Concatenate the latent vector with the embedding
    edit_vec = net(x_a, embed_ab)

    # Add the edit vector to the latent vector
    #return edit_vec
    return edit_vec + x_a

def apply_edit__oracle_mag(net, x_a, x_b, embed_ab):
    # Reshape from (B, D, K) to (B, M)
    x_a = x_a.flatten(1)
    embed_ab = embed_ab.flatten(1)

    # Concatenate the latent vector with the embedding
    edit_dir, magnitude = net.forward_decoupled(x_a, embed_ab)
    gt_vec = (x_b-x_a)
    opt_magnitude = torch.zeros_like(magnitude)
    for i in range(x_a.shape[0]):
        opt_magnitude[i] = solve_eq(a=edit_dir[i], b=gt_vec[i])
    edit_vec = edit_dir * opt_magnitude

    # Add the edit vector to the latent vector
    return edit_vec + x_a

def apply_edit__oracle_dir(net, x_a, x_b, embed_ab):
    # Reshape from (B, D, K) to (B, M)
    x_a = x_a.flatten(1)
    embed_ab = embed_ab.flatten(1)

    # Concatenate the latent vector with the embedding
    edit_dir, magnitude = net.forward_decoupled(x_a, embed_ab)
    gt_vec = (x_b-x_a)
    opt_direction = torch.zeros_like(edit_dir)
    for i in range(x_a.shape[0]):
        opt_direction[i] = gt_vec[i] / (torch.norm(gt_vec[i]) + 1e-8)

    # Add the edit vector to the latent vector
    return opt_direction*magnitude + x_a

def solve_eq(a, b):
    """
    Minimize ||x*a - b||^2, where x is a scalar, and a and b are fixed vectors.
    """
    return (b * a).sum() / (a * a).sum()

def apply_iterated_edits(model, embeds_a, embeds_b, embeds_text):
    # Move all the garbage to CUDA
    embeds_a = embeds_a.cuda()
    embeds_b = embeds_b.cuda()
    embeds_text = embeds_text.cuda()

    x_b_edited = apply_edit__oracle_dir(model, embeds_a, embeds_b, embeds_text)
    x_b = embeds_b
    x_a = embeds_a

    # Decode the batch
    b_size = x_b.shape[0]

    with torch.inference_mode():
        orig = pc_ae.decoder(x_a).reshape([b_size, 4096, 3])
        rec = pc_ae.decoder(x_b_edited).reshape([b_size, 4096, 3])
        rec_gt = pc_ae.decoder(x_b).reshape([b_size, 4096, 3])

    return (orig, rec, rec_gt), (x_a, x_b_edited, x_b)


In [9]:
model = model.eval()
metric_meter = MetricLogger()

def get_metrics(data_loader):
    with torch.no_grad():
        chain_count = 0
        for batch_k, (chain_ids, edit_keys, node_a, node_b, text_embeds) in enumerate(data_loader):
            #print(chain_ids)
            if batch_k == len(data_loader_val) - 1 - drop_n:
                break
            if chain_count == 0:
                prev_node = node_a
            
            # Apply the edits
            (orig, rec, rec_gt), (x_a, x_b_edited, x_b) = apply_iterated_edits(model,
                                                                               embeds_a=prev_node,
                                                                               embeds_b=node_b,
                                                                               embeds_text=text_embeds)
    
            # Compute average pairwise L2 distance in feature space
            l2_dist = torch.norm(x_b_edited - x_b, p=2)
            l2_scale = torch.norm(x_b_edited - x_a, p=2)
    
            # Compute average pairwise CD
            cd_dist = chamfer_loss(rec, rec_gt, reduction="mean").mean()
            cd_scale = chamfer_loss(rec, orig, reduction="mean").mean()
    
            metric_meter.update(avg_l2_dist=l2_dist.item(),
                                avg_l2_scale=l2_scale.item(),
                                cd_dist=cd_dist.item(),
                                cd_scale=cd_scale.item())
    
            prev_node = x_b_edited
    
            chain_count += 1
            if chain_count == args.chain_length:
                metric_meter.update(final_l2_dist=l2_dist.item(),
                                    final_l2_scale=l2_scale.item(),
                                    final_cd_dist=cd_dist.item(),
                                    final_cd_scale=cd_scale.item())
                chain_count = 0

    return metric_meter

avg_fcd = []
avg_acd = []
avg_fl2 = []
avg_al2 = []
for chain_length, batch_size, drop_n in [[10, 8, 0], [15, 4, 3], [20, 3, 0]]:
    #print("chain_length=", chain_length, "batch_size=", batch_size)
    data_loader_val = get_loader(args, batch_size=batch_size, chain_length=chain_length)
    metric_meter = get_metrics(data_loader_val)
    
    if chain_length==10:
        print(args.model)
        print("======"*10)

    # print("|P| = %d" % chain_length)
    table_str = "& & FCD & \multicolumn{1}{c}{ACD} & \multicolumn{1}{c}{FL2} & \multicolumn{1}{c}{AL2} "
    table_str = table_str.replace("FCD", "%0.3f" % round(metric_meter.final_cd_dist.global_avg*10**4, 3))
    table_str = table_str.replace("ACD", "%0.3f" % round(metric_meter.cd_dist.global_avg*10**4, 3))
    
    table_str = table_str.replace("FL2", "%0.3f" % round(metric_meter.final_l2_dist.global_avg, 3))
    table_str = table_str.replace("AL2", "%0.3f" % round(metric_meter.avg_l2_dist.global_avg, 3))
    avg_fcd += [metric_meter.final_cd_dist.global_avg*10**4]
    avg_acd += [metric_meter.cd_dist.global_avg*10**4]
    avg_fl2 += [metric_meter.final_l2_dist.global_avg]
    avg_al2 += [metric_meter.avg_l2_dist.global_avg]

    print(table_str)
print("\\\\")
print()
print("& \multicolumn{1}{c}{\icono} & & %0.3f & %0.3f & %0.3f & %0.3f \\" % (np.mean(avg_fcd), np.mean(avg_acd), np.mean(avg_fl2), np.mean(avg_al2)))

/home/slimhy/conda/envs/shape2vecset/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


mlp_mapper_bert_l8_pcae
& & 34.114 & \multicolumn{1}{c}{21.346} & \multicolumn{1}{c}{7.912} & \multicolumn{1}{c}{6.539} 
& & 27.319 & \multicolumn{1}{c}{21.362} & \multicolumn{1}{c}{6.447} & \multicolumn{1}{c}{5.299} 
& & 24.498 & \multicolumn{1}{c}{17.756} & \multicolumn{1}{c}{5.798} & \multicolumn{1}{c}{4.452} 
\\

& \multicolumn{1}{c}{\icono} & & 28.644 & 20.155 & 6.719 & 5.430 \
